03 Feature Engineering<div style="font-family: system-ui, -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif; border: 1px solid #ddd; border-radius: 10px; padding: 16px 20px; margin-bottom: 16px; background: #fafafa;">
  <h1 style="margin-top: 0; margin-bottom: 8px; font-size: 24px;">HAVI – 03_feature_engineering</h1>
  <p style="margin: 0 0 12px 0; font-size: 14px;">
    Cel: na bazie zbioru <code>master_clean</code> zbudować cechy kalendarzowe i czasowe (lagi, średnie kroczące, sezony, flagi świąteczne),
    przygotować tabelę <code>master_model_ready</code> do trenowania modeli prognozujących.
  </p>
</div>


HAVI – 03_feature_engineering

Cel:
- Wczytać prepared/series_level_a.parquet + series_registry.csv (z etapu 02)
- Pracować na Level A: country + sku
- Zbudować spójny zbiór cech do modeli global ML (jedna tabela: target + features)
- Zapewnić ciągłość osi czasu (per seria) poprzez jawne uzupełnienie braków tygodniowych
  (domyślnie: brak tygodnia = 0 popytu, bo pracujemy na agregacji shipments/sales)

Output:
- data/features/features_level_a.parquet (target + cechy)


### Load danych i kontrola spójności

Wczytujemy dane, zakładamy, że struktura danych jest poprawna i wykonujemy jedynie podstawową kontrolę spójności.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", None)

BASE_DIR = Path(".")
DATA_DIR = BASE_DIR / "data"
PREP_DIR = DATA_DIR / "prepared"

FEAT_DIR = DATA_DIR / "features"
FEAT_DIR.mkdir(exist_ok=True, parents=True)

LEVELA_PATH = PREP_DIR / "series_level_a.parquet"
REG_PATH = PREP_DIR / "series_registry.csv"

LEVELA_PATH, REG_PATH


(WindowsPath('data/prepared/series_level_a.parquet'),
 WindowsPath('data/prepared/series_registry.csv'))

### Wybór serii do modelowania

Dołączamy metryki jakości serii oraz flagę `eligible`.
Dalsze przetwarzanie dotyczy wyłącznie serii spełniających kryteria modelowe.


In [2]:
df = pd.read_parquet(LEVELA_PATH)
reg = pd.read_csv(REG_PATH)

df["week_start"] = pd.to_datetime(df["week_start"], errors="coerce")
df["country"] = df["country"].astype(str).str.strip()
df["sku"] = df["sku"].astype(str).str.strip()
df["product_name"] = df["product_name"].astype(str).str.strip()

reg["country"] = reg["country"].astype(str).str.strip()
reg["sku"] = reg["sku"].astype(str).str.strip()

display(df.head())
display(reg.head())

assert df["week_start"].isna().sum() == 0
assert df["demand"].isna().sum() == 0


,country,sku,week_start,demand,product_name,n_dc,is_outlier
0,Germany,00004-807-019,2022-01-03,7233.0,Pommes Frites I,1,False
1,Germany,00004-807-019,2022-01-10,5271.0,Pommes Frites I,1,False
2,Germany,00004-807-019,2022-01-17,5462.0,Pommes Frites I,1,False
3,Germany,00004-807-019,2022-01-24,6225.0,Pommes Frites I,1,False
4,Germany,00004-807-019,2022-01-31,5095.0,Pommes Frites I,2,False


,country,sku,longest_gap_weeks,missing_weeks,n_segments,n_weeks_obs,span_weeks,zero_share,n_nonzero,avg_gap_nonzero,ADI,CV2,demand_type,eligible
0,Germany,00004-807-019,8,13,4,188,201,0.0,188.0,1.0,1.0,0.122573,smooth,False
1,Germany,00019-003-003,0,0,1,202,202,0.0,202.0,1.0,1.0,0.035450,smooth,True
2,Poland,02589-489-000,1,1,2,357,358,0.0,357.0,1.0,1.0,0.037047,smooth,True
3,Poland,05243-022-000,15,20,5,338,358,0.0,338.0,1.0,1.0,0.100955,smooth,False
4,Poland,16333-000-000,8,9,3,205,214,0.0,205.0,1.0,1.0,0.279117,smooth,False


### Ujednolicenie osi czasu

Modele uczące się na lagach wymagają stałej siatki czasowej.
Dla każdej serii budujemy pełny kalendarz tygodniowy.


In [3]:
df = df.merge(
    reg[["country","sku","eligible","demand_type","zero_share","n_weeks_obs","longest_gap_weeks","missing_weeks"]],
    on=["country","sku"],
    how="left"
)

assert df["eligible"].isna().sum() == 0, "Braki eligible — registry nie pasuje do Level A."

df_elig = df[df["eligible"]].copy()

print("Rows total:", len(df))
print("Rows eligible:", len(df_elig))
print("Series eligible:", df_elig[["country","sku"]].drop_duplicates().shape[0])


Rows total: 5810
Rows eligible: 4258
Series eligible: 15


### Obsługa brakujących tygodni

Brakujące tygodnie są jawnie uzupełniane zerowym popytem.
Decyzja ta jest świadomym założeniem modelowym dla danych zagregowanych tygodniowo.


In [4]:
SERIES_KEY = ["country","sku"]
TIME_COL = "week_start"
TARGET = "demand"

def reindex_weekly_full(g: pd.DataFrame) -> pd.DataFrame:
    g = g.sort_values(TIME_COL).copy()
    full = pd.date_range(g[TIME_COL].min(), g[TIME_COL].max(), freq="W-MON")
    g = g.set_index(TIME_COL).reindex(full)
    g.index.name = TIME_COL
    g = g.reset_index()

    # przy reindex tracimy klucze serii → odtwarzamy
    for k in SERIES_KEY:
        g[k] = g[k].iloc[0]

    # target: brak tygodnia => 0
    g[TARGET] = g[TARGET].fillna(0.0)

    # metadane: product_name i n_dc bierzemy z pierwszego nie-NA w oryginale
    # (jeśli po reindexu są NA w pierwszych wierszach)
    if "product_name" in g.columns:
        g["product_name"] = g["product_name"].ffill().bfill()
    if "n_dc" in g.columns:
        g["n_dc"] = g["n_dc"].ffill().bfill()

    # flagi: czy ten punkt był imputowany (przydatne jako feature)
    g["was_missing_week"] = (g[TARGET] == 0.0)  # uproszczenie
    return g

df_full = (
    df_elig
    .groupby(SERIES_KEY, observed=True, group_keys=False)
    .apply(reindex_weekly_full)
    .reset_index(drop=True)
)

df_full.head(10), df_full.shape



C:\Users\48573\AppData\Local\Temp\ipykernel_3472\1590237095.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(reindex_weekly_full)


(  week_start  country            sku  demand        product_name  n_dc  \
 0 2021-12-27  Germany  00019-003-003    28.0  Coca Cola 20 L BIB   3.0   
 1 2022-01-03  Germany  00019-003-003  9732.0  Coca Cola 20 L BIB   8.0   
 2 2022-01-10  Germany  00019-003-003  9002.0  Coca Cola 20 L BIB   8.0   
 3 2022-01-17  Germany  00019-003-003  7403.0  Coca Cola 20 L BIB   8.0   
 4 2022-01-24  Germany  00019-003-003  7870.0  Coca Cola 20 L BIB   8.0   
 5 2022-01-31  Germany  00019-003-003  7722.0  Coca Cola 20 L BIB   8.0   
 6 2022-02-07  Germany  00019-003-003  8214.0  Coca Cola 20 L BIB   8.0   
 7 2022-02-14  Germany  00019-003-003  8286.0  Coca Cola 20 L BIB   8.0   
 8 2022-02-21  Germany  00019-003-003  8166.0  Coca Cola 20 L BIB   8.0   
 9 2022-02-28  Germany  00019-003-003  6992.0  Coca Cola 20 L BIB   8.0   
 
   is_outlier eligible demand_type  zero_share  n_weeks_obs  longest_gap_weeks  \
 0       True     True      smooth         0.0        202.0                0.0   
 1      F

### Cechy kalendarzowe i czasowe

Tworzymy podstawowe cechy opisujące pozycję w czasie oraz cykliczność roku.
Zapewnia to modelom informację o sezonowości i trendzie.


In [5]:
df_full["year"] = df_full[TIME_COL].dt.isocalendar().year.astype(int)
df_full["week"] = df_full[TIME_COL].dt.isocalendar().week.astype(int)

# indeks czasu per seria
df_full["time_idx"] = (
    df_full
    .groupby(SERIES_KEY, observed=True)[TIME_COL]
    .rank(method="dense")
    .astype(int) - 1
)

# cykliczność tygodnia
w = df_full["week"].astype(float)
df_full["week_sin"] = np.sin(2 * np.pi * w / 52.0)
df_full["week_cos"] = np.cos(2 * np.pi * w / 52.0)

# proste flagi sezonowe (opcjonalne, często pomagają)
df_full["is_q4"] = df_full["week"].between(40, 53).astype(int)
df_full["is_summer"] = df_full["week"].between(22, 35).astype(int)

display(df_full[[*SERIES_KEY, TIME_COL, "year", "week", "time_idx", "week_sin", "week_cos"]].head(10))


,country,sku,week_start,year,week,time_idx,week_sin,week_cos
0,Germany,00019-003-003,2021-12-27,2021,52,0,6.432491e-16,1.000000
1,Germany,00019-003-003,2022-01-03,2022,1,1,1.205367e-01,0.992709
2,Germany,00019-003-003,2022-01-10,2022,2,2,2.393157e-01,0.970942
3,Germany,00019-003-003,2022-01-17,2022,3,3,3.546049e-01,0.935016
4,Germany,00019-003-003,2022-01-24,2022,4,4,4.647232e-01,0.885456
5,Germany,00019-003-003,2022-01-31,2022,5,5,5.680647e-01,0.822984
6,Germany,00019-003-003,2022-02-07,2022,6,6,6.631227e-01,0.748511
7,Germany,00019-003-003,2022-02-14,2022,7,7,7.485107e-01,0.663123
8,Germany,00019-003-003,2022-02-21,2022,8,8,8.229839e-01,0.568065
9,Germany,00019-003-003,2022-02-28,2022,9,9,8.854560e-01,0.464723


### Walidacja struktury czasowej

Sprawdzamy, czy każda seria posiada dokładnie jedną obserwację na tydzień.
Na tym etapie dane są gotowe do budowy cech opartych o historię.


In [6]:
check = (
    df_full
    .groupby(SERIES_KEY, observed=True)
    .agg(
        min_date=(TIME_COL, "min"),
        max_date=(TIME_COL, "max"),
        n_rows=(TIME_COL, "size"),
        n_unique=(TIME_COL, "nunique"),
        sum_target=(TARGET, "sum"),
    )
    .reset_index()
)

assert (check["n_rows"] == check["n_unique"]).all(), "Są duplikaty po reindex — powinno być 1 wiersz na tydzień."
display(check.head(10))


,country,sku,min_date,max_date,n_rows,n_unique,sum_target
0,Germany,00019-003-003,2021-12-27,2025-11-03,202,202,1822324.000
1,Poland,02589-489-000,2018-12-31,2025-11-03,358,358,2706618.500
2,Poland,62170-027-000,2022-05-23,2025-11-03,181,181,11755.600
3,Portugal,00012-619-000,2021-03-01,2025-11-03,245,245,1690936.817
4,Portugal,00041-097-000,2018-12-31,2025-11-03,358,358,9592.344
5,Romania,00012-432-000,2018-12-31,2025-10-27,357,357,297001.350
6,Romania,00077-010-000,2018-12-31,2025-10-27,357,357,153253.504
7,Romania,05243-115-000,2023-07-31,2025-10-27,118,118,23623.503
8,Romania,07808-016-000,2019-07-01,2025-10-27,331,331,878.000
9,Spain,00023-189-000,2018-12-31,2025-11-03,358,358,74393.922


### Checkpoint danych cechowych

Zapisujemy pośrednią wersję zbioru cech.
Umożliwia to szybki powrót do tego etapu bez ponownego przeliczania osi czasu.


In [7]:
CHK_PATH = FEAT_DIR / "features_level_a_checkpoint_part1.parquet"
df_full.to_parquet(CHK_PATH, index=False)
CHK_PATH


WindowsPath('data/features/features_level_a_checkpoint_part1.parquet')

### Rozróżnienie braków i rzeczywistych zer

Odróżniamy tygodnie rzeczywiście obecne w danych od tych,
które zostały uzupełnione podczas reindexowania osi czasu.


In [8]:
# flaga czy tydzień istniał w danych źródłowych
orig_keys = df_elig[["country","sku","week_start"]].assign(existed=1)

df_full = df_full.merge(
    orig_keys,
    on=["country","sku","week_start"],
    how="left"
)

df_full["was_missing_week"] = df_full["existed"].isna()
df_full["existed"] = df_full["existed"].fillna(0).astype(int)


### Opóźnienia zmiennej docelowej

Tworzymy opóźnienia popytu, które stanowią podstawowe sygnały predykcyjne
dla modeli opartych o historię.


In [9]:
LAGS = [1, 2, 4, 8, 13, 26, 52]

for lag in LAGS:
    df_full[f"lag_{lag}"] = (
        df_full
        .groupby(["country","sku"], observed=True)["demand"]
        .shift(lag)
    )



### Statystyki kroczące

Wyznaczamy zagregowane miary poziomu i zmienności popytu
w różnych horyzontach czasowych.


In [10]:
WINDOWS = [4, 8, 13, 26, 52]

for w in WINDOWS:
    df_full[f"roll_mean_{w}"] = (
        df_full
        .groupby(["country","sku"], observed=True)["demand"]
        .shift(1)
        .rolling(w)
        .mean()
    )

    df_full[f"roll_std_{w}"] = (
        df_full
        .groupby(["country","sku"], observed=True)["demand"]
        .shift(1)
        .rolling(w)
        .std()
    )



### Czas od ostatniego popytu

Cechy opisujące czas od ostatniego niezerowego wolumenu
są kluczowe dla serii z popytem nieregularnym.


In [11]:
def weeks_since_last_nonzero(s: pd.Series) -> pd.Series:
    last = -1
    out = []
    for i, v in enumerate(s):
        if v > 0:
            last = i
            out.append(0)
        else:
            out.append(np.nan if last == -1 else i - last)
    return pd.Series(out, index=s.index)

df_full["weeks_since_nonzero"] = (
    df_full
    .groupby(["country","sku"], observed=True)["demand"]
    .apply(weeks_since_last_nonzero)
    .reset_index(level=[0,1], drop=True)
)


### Flagi wspomagające model

Dodajemy proste flagi logiczne,
które pozwalają modelowi rozróżnić różne stany serii.


In [12]:
df_full["is_zero"] = (df_full["demand"] == 0).astype(int)

df_full["is_outlier"] = (
    df_full["is_outlier"]
    .fillna(False)
    .astype(int)
)


C:\Users\48573\AppData\Local\Temp\ipykernel_3472\2759542554.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(False)


### Usunięcie niepełnych obserwacji

Pierwsze obserwacje nie posiadają pełnej historii opóźnień
i nie są użyteczne w treningu modeli.


In [13]:
min_lag = max(LAGS)
df_model = df_full[df_full["time_idx"] >= min_lag].copy()


### Finalny zestaw cech

Definiujemy zbiór kolumn,
który będzie używany spójnie we wszystkich modelach.


In [14]:
FEATURE_COLS = (
    [f"lag_{l}" for l in LAGS] +
    [f"roll_mean_{w}" for w in WINDOWS] +
    [f"roll_std_{w}" for w in WINDOWS] +
    [
        "week_sin",
        "week_cos",
        "time_idx",
        "weeks_since_nonzero",
        "was_missing_week",
        "is_zero",
        "is_outlier",
    ]
)



### Zapis danych do modelowania

Zapisujemy finalny zbiór cech,
który stanowi wejście do treningu i backtestingu modeli.


In [15]:
FINAL_PATH = FEAT_DIR / "features_level_a.parquet"
df_model.to_parquet(FINAL_PATH, index=False)

FINAL_PATH


WindowsPath('data/features/features_level_a.parquet')

### Podsumowanie etapu feature engineering

Zbudowano spójny zbiór cech oparty o historię popytu,
czas oraz sezonowość.
Dane są gotowe do backtestingu i treningu modeli prognostycznych.
